Лихобаба Даниил <br>
Б03-903 <br>
likhobaba.dp@phystech.edu

# Apriori алгоритм в ARL

### Загрузка данных

In [1]:
import pandas as pd
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Заполним пропуски последним предметом для удобства дальнейшей обработки

In [2]:
dataset.fillna(method='ffill', axis=1, inplace=True)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs,eggs
2,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney,chutney
3,turkey,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado,avocado
4,mineral water,milk,energy bar,whole wheat rice,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea,green tea


Сделаем из датасета матрицу транзакций (просто перенесём данные из `DataFrame` в `List`)

In [3]:
transactions = []
for i in range(dataset.shape[0]):
    transactions.append([dataset.values[i, j] for j in range(dataset.shape[1])])

### Применение алгоритма

In [4]:
def num_after_point(num):
    try:
        return len(str(num).split('.')[1])
    except IndexError:
        return 0

In [5]:
from apyori import apriori
import apyori
min_support = 0.003
min_confidence = 0.2
min_lift = 5
min_length = 2
result = list(apriori(transactions, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift, 
                      min_length=min_length))

Преобразуем данные для визуализации и визуализируем

In [6]:
import shutil, os, json
from io import StringIO
output = []
for rl in result:
    o = StringIO()
    apyori.dump_as_json(rl, o)
    info = json.loads(o.getvalue())
    info['support'] = round(info['support'], num_after_point(min_support)+2)
    for i in info['ordered_statistics']:
        i['confidence'] = round(i['confidence'], num_after_point(min_confidence)+2)
        i['lift'] = round(i['lift'], num_after_point(min_lift)+2)
    output.append(info)
# for i in output:
#     print(i, '\n')
data_df = pd.DataFrame(output)
pd.set_option('display.max_colwidth', None)

from IPython.display import display, HTML
display(HTML(data_df.to_html()))

,items,support,ordered_statistics
0,"[fromage blanc, honey]",0.00333,"[{'items_base': ['fromage blanc'], 'items_add': ['honey'], 'confidence': 0.245, 'lift': 5.16}]"
1,"[ground beef, spaghetti, tomato sauce]",0.00307,"[{'items_base': ['tomato sauce'], 'items_add': ['ground beef', 'spaghetti'], 'confidence': 0.217, 'lift': 5.54}]"
2,"[mineral water, olive oil, whole wheat pasta]",0.00387,"[{'items_base': ['mineral water', 'whole wheat pasta'], 'items_add': ['olive oil'], 'confidence': 0.403, 'lift': 6.12}]"
3,"[frozen vegetables, milk, mineral water, olive oil]",0.00333,"[{'items_base': ['frozen vegetables', 'olive oil'], 'items_add': ['milk', 'mineral water'], 'confidence': 0.294, 'lift': 6.13}]"
4,"[frozen vegetables, milk, mineral water, soup]",0.00307,"[{'items_base': ['frozen vegetables', 'soup'], 'items_add': ['milk', 'mineral water'], 'confidence': 0.383, 'lift': 7.99}, {'items_base': ['milk', 'soup'], 'items_add': ['frozen vegetables', 'mineral water'], 'confidence': 0.202, 'lift': 5.65}, {'items_base': ['frozen vegetables', 'milk', 'mineral water'], 'items_add': ['soup'], 'confidence': 0.277, 'lift': 5.48}]"
